# Indexing

Alle Daten in die Datenbank bringen. Ich verwende ChromaDB weil lokal. Es ist die Grundlage für das spätere System und gleichzeitig die Quelle für die Evaluation.

In [12]:
import chromadb
import random
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer


## Embeddings laden

In [3]:
embeddings = np.load('../2-embedding/embeddings_gbert.npy')
chunks_meta = pd.read_json('../2-embedding/chunks_metadata.jsonl', lines=True)

## DB und Collection erstellen

In [4]:
client = chromadb.PersistentClient(path="./chroma_db")

collection = client.get_or_create_collection(
    name = "prdukt_chunks",
    metadata = {"description": "Product chunks with GBERT embeddings"}
)

## Daten vorverarbeiten

In [5]:
ids = [str(i) for i in range(len(chunks_meta))]
documents = chunks_meta['document'].tolist()
metadatas = chunks_meta['metadata'].tolist()
embeddings_list = embeddings.tolist()

## Indexieren

In [7]:
collection.add(
    ids = ids,
    documents = documents,
    embeddings = embeddings_list,
    metadatas = metadatas
)

print(f"Done")

Done


## Check DB

In [18]:
print(f"Anzahl der Dokumente: {collection.count()}")

sample_idx = [str(i) for i in random.sample(range(len(embeddings)), int(len(embeddings) * 0.005))]

result = collection.get(
    ids = sample_idx,
    include = ["documents", "metadatas", "embeddings"]
)

for i, (doc, meta) in enumerate(zip(result['documents'], result["metadatas"])):
    print(f"--- Dokuemnt {i} ---")
    print(f"Text: {doc[:200]}")
    print(f"Metadata: {meta}")

Anzahl der Dokumente: 1800
--- Dokuemnt 0 ---
Text: Der Kirsch MED-126 PRO-ACTIVE ist ein Medikamentenkühlschrank mit 120 Litern Nutzvolumen und einer Temperaturregelung von +2 bis +14°C. Der Kühlschrank entspricht der DIN 58345-Norm und ist ideal für 
Metadata: {'chunk_id': 'Kirsch-MED-126-PRO-ACTIVE-Medikamentenkuehlschrank_summ_0', 'product_manufacturer': 'Kirsch', 'chunk_type': 'description', 'product_id': 'Kirsch-MED-126-PRO-ACTIVE-Medikamentenkuehlschrank', 'product_url': 'https://www.rainer-medizintechnik.de/Kirsch-MED-126-PRO-ACTIVE-Medikamentenkuehlschrank', 'product_price': '2.650,38\xa0€', 'product_sku': 'KIR 12136', 'product_category': 'Medikamentenkühlschrank', 'product_title': 'MED-126 PRO-ACTIVE'}
--- Dokuemnt 1 ---
Text: Die Isolierung des FROSTER MED-95 ist 50 mm stark und besteht aus hochwertigem, druckgeschäumtem und umweltschonendem Material. Der Innenraum ist aus schlagfestem, weißem Kunststoff mit eingeformten A
Metadata: {'product_sku': 'KIR 10304', 'chunk_id': '